In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
train_dir = "/Users/sreyakaruturi/Downloads/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/train"
test_dir = "/Users/sreyakaruturi/Downloads/FOREST_FIRE_SMOKE_AND_NON_FIRE_DATASET/test"

In [4]:
def load_images(directory, batch_size=1024):
    images = []
    labels = []
    label_encoder = LabelEncoder()

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            files = os.listdir(label_path)
            random.shuffle(files)  # Shuffle the files to load a random subset
            for image_file in files[:batch_size]:
                image_path = os.path.join(label_path, image_file)
                # Load and preprocess images as needed
                img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
                img = tf.keras.preprocessing.image.img_to_array(img)
                img = tf.keras.applications.resnet50.preprocess_input(img)

                images.append(img)
                labels.append(label)

    labels = label_encoder.fit_transform(labels)

    return np.array(images), np.array(labels)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    # Specify your preprocessing options here
)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=1024,  # Adjust batch size as needed
    class_mode='sparse'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=1024,
    class_mode='sparse'
)

Found 32398 images belonging to 3 classes.
Found 10500 images belonging to 3 classes.


In [6]:
X_train, y_train = load_images(train_dir)
X_test, y_test = load_images(test_dir)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

In [7]:


def create_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))  

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = X_train[0].shape
cnn_model = create_cnn_model(input_shape)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
history_transfer_learning = cnn_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping])
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"CNN Model Accuracy: {cnn_accuracy}")

Epoch 1/20
77/77 [==============================] - 56s 714ms/step - loss: 23.6500 - accuracy: 0.7806 - val_loss: 0.3906 - val_accuracy: 0.8602
Epoch 2/20
77/77 [==============================] - 29s 376ms/step - loss: 0.3478 - accuracy: 0.8820 - val_loss: 0.3245 - val_accuracy: 0.8862
Epoch 3/20
77/77 [==============================] - 27s 356ms/step - loss: 0.2387 - accuracy: 0.9259 - val_loss: 0.4682 - val_accuracy: 0.8098
Epoch 4/20
77/77 [==============================] - 34s 439ms/step - loss: 0.2044 - accuracy: 0.9430 - val_loss: 0.3902 - val_accuracy: 0.9089
Epoch 5/20
77/77 [==============================] - 48s 624ms/step - loss: 0.1121 - accuracy: 0.9666 - val_loss: 0.3740 - val_accuracy: 0.9041
Epoch 6/20
77/77 [==============================] - 47s 615ms/step - loss: 0.0746 - accuracy: 0.9796 - val_loss: 0.7179 - val_accuracy: 0.8780
Epoch 7/20
77/77 [==============================] - 46s 597ms/step - loss: 0.0565 - accuracy: 0.9829 - val_loss: 0.6517 - val_accuracy: 0.902